# Problem 1 Initial Simulations

TJ Kim

Date: 3/25/21
Updated: 3/25/21

#### Summary:

Using the User, Server, and Job classes from the MEC project, generate a sandbox in order to make the matching problem possible.

In [1]:
cd /home/tkim/CMU/DistMEC

/home/tkim/CMU/DistMEC


Import relevant libraries

In [2]:
import numpy as np
import math

from classes.Application import *
from classes.User import *
from classes.Server import *
from solver.Sim_Params import *
from classes.Central_Controller import *

### Central Controller Class

Draft the class here before separating in own py file.

In [200]:
import numpy as np
import copy

class Central_Controller:
    """
    Central controller solves P1 by placing VMs at desired servers
    Keeps track of latency based weights for users and inits them for VMs without history
    """
    
    def __init__(self, servers, containers, system_params):
        
        """
        All inputs are lists of objects of their relevant type
        """
        
        self.servers = servers
        self.containers = containers
        
        # Extract information
        self.user_locations = None
        self.beta_weights = None
        
        # System settings
        self.ts_big = 0
        self.ts_small = 0
        
        # Extract from System Params
        self.num_app = system_params.num_app
        self.num_user = system_params.num_user
        self.num_servers = len(self.servers)
        self.num_cores = self.obtain_num_cores()
        self.max_deployed = self.obtain_max_deployed()
        
        # Initialize Utilities
        # self.container_utility = np.zeros(len(containers))
        # self.app_utility = np.zeros(self.num_app)
        self.container_deployed = np.zeros([self.num_app, len(self.servers)]) # 1- True, 0 - False
        
        
    def update_big_ts(self):
        
        self.ts_big += 1
        
    def update_small_ts(self):
        
        self.ts_small += 1
        
    def obtain_num_cores(self):
        """
        Get the total number of VMs that can be deployed in the system
        on a per server basis
        """
        
        num_cores = np.ones(len(servers))
        
        for i in range(len(servers)):
            server = servers[i]
            num_cores[i] = int(server.avail_rsrc[0])
            
        return num_cores
    
    def obtain_max_deployed(self):
        """
        obtain the maximum allowed unique applications for each server?
        """
        
        num_cores = np.ones(len(servers))
        for i in range(len(servers)):
            server = servers[i]
            num_cores[i] = min(server.avail_rsrc[0],self.num_app)
            
        return num_cores
        
        
    def VM_placement(self, users, apps, limit = None):
        """
        solves P1 - The VM placement problem at every big TS
        inputs: users-apps are lists of nominal objects
                limit - number of iterations to run swapping before quitting
        """
        
        # Flush utility functions
        self.container_utility = np.zeros(len(containers))
        self.app_utility = np.zeros(self.num_app)
        self.container_deployed = np.zeros([len(servers),num_app]) 
        
        # Initialize the Container placements on VMs 
        self.container_deployed = VM_placement_init(apps)
        
        # Calculate utility function for all placed containers and apps
        
        
        # Sort utility functions for both app/containers
        
        # Matching Iteratively - from lowest app --> container and swap
        
        return
        
    def VM_placement_init(self, apps):
        """
        Set aside proportional vm counts for app load
        Place them closest to most users in round robin style
        """
        
        # Get expected load of each application group
        load_product = np.zeros(self.num_app)
        
        for app in apps:
            job_type = app.job_type
            load_product[job_type] += app.offload_mean
                
        # Calculate number of VM for each app
        container_deployed = np.zeros([self.num_app, self.num_servers])
        space_available = copy.deepcopy(self.max_deployed)
        total_VM = np.sum(self.max_deployed)
        
        load_prop = load_product/np.sum(load_product)
        init_deployed = np.floor(load_prop * (total_VM - self.num_app)) + 1
        
        # Cut all init deployed below num_server
        edit_idx = np.argwhere(init_deployed>len(self.servers))
        init_deployed[edit_idx] = len(self.servers)
        
        # Deploy the servers from max-->min count
        deploy_order = np.argsort(load_prop)
        
        for app_idx in deploy_order:
            still_avail = np.subtract(space_available, np.sum(container_deployed,axis=0))
            avail_servers = np.argwhere(still_avail>0).flatten()
            num_VM = int(min(init_deployed[app_idx],avail_servers.shape[0]))
            
            deploy_servers = np.random.choice(avail_servers, size=num_VM, replace=False, p=None)
            container_deployed[app_idx,deploy_servers] = 1
            
        # Fill in the available cores with empty VMs
        still_avail = np.subtract(space_available, np.sum(container_deployed,axis=0))
        avail_s_idx = np.argwhere(still_avail>0).flatten()
                
        for s in avail_s_idx:
            deployed = container_deployed[:,s]
            candidates = np.argwhere(deployed == 0).flatten()            
            new_apps = np.random.choice(candidates,size= int(still_avail[s]),replace=False,p=None)
            
            container_deployed[new_apps,s] = 1
            
        
        return container_deployed
        
    def compute_container_utility(self, users, apps, container):
        """
        Compute all utilities for containers that are deployed
        """
        
        return
        
    def compute_app_utility(self, users, apps, containers):
        """
        Compute utilities for all apps given 
        AKA sum all the utilities for all apps 
        """
        
        return
    


### Generate Servers, Users, and Jobs.

First make simulation parameters and job profiles (apps).

In [201]:
"""
Job Profiles - 3 apps
"""

num_app_types = 8
low_mean = 1
high_mean = 3
job_profiles = []

for i in range(num_app_types):
    job_profiles += [Job_Profile(job_name = str(i),
                                 latency_req = 1e-5,
                                 offload_mean = np.random.uniform(low_mean,high_mean))]
    
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps = 3, x_length = 1, y_length = 1)
boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])
sim_param.num_app = num_app_types
sim_param.num_user = 30

Make the servers, users, apps

In [207]:
# Make Servers
# Server Settings
num_server_l1 = 0
num_server_l2 = 10
num_server_l3 = 0

num_resource = 1
weak_range = np.array([[1,4]])
strong_range = np.array([[1,4]])

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    idx_counter += 1
    
    
servers = servers_l2

"""
Make Users
"""

# User Settings
num_user_m0 = 0 # Pedestrian
num_user_m1 = 0 # Public Transport
num_user_m2 = 30 # Vehicle

max_speed = 2.5
lamdas = [1/0.25,1/0.83,1/1.67] # 3 mph, 10 mph, 20 mph
num_path = 10

# Generate Server
users_m0 = []
users_m1 = []
users_m2 = []
idx_counter = 0


for i in range(num_user_m2):
    users_m2 += [User(boundaries, sim_param.time_steps, 2, lamdas, max_speed)]
    users_m2[-1].generate_MC(servers)
    users_m2[-1].assign_id(idx_counter)
    idx_counter += 1

users = users_m2

"""
Make Apps
"""

num_apps = len(users)
app_id = np.random.choice(num_app_types,num_apps)
apps = []

for i in range(len(app_id)):
    apps += [Application(job_type=app_id[i], user_id=i, 
                         time_steps=sim_param.time_steps, job_profiles=job_profiles)]



### Testing Central Controller App

Initialize the central controller.

In [208]:
cc = Central_Controller(servers=servers, containers=None, system_params=sim_param)

In [209]:
init = cc.VM_placement_init(apps)
init

array([[1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0., 1., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [210]:
cc.max_deployed

array([3., 1., 2., 1., 3., 2., 1., 1., 3., 2.])

In [211]:
np.sum(init,axis=0)

array([3., 1., 2., 1., 3., 2., 1., 1., 3., 2.])